- Lấy ra middlename.
- Tính TF-IDF của các middlename này.

In [6]:
import time
import re
import nltk
import numpy as np
import pandas as pd

In [7]:
# Loading dataset
file = pd.read_csv("name.csv", encoding='utf-8')

In [8]:
# Số lượng dòng, cột của file csv
print(file.shape)

(100, 2)


In [9]:
name = file['full_name']
label = file['gender']

In [13]:
# Tiền xử lý dữ liệu cho full name
male_names = ''
female_names = ''
words = 0

for sen in range(0,len(name)):
    
    # Dòng này đếm số lượng ký tự
    #words += len(str(name[sen]).split())
    
    if (label[sen] == 1) and (len(str(name[sen]).split()) > 2):
        # Remove all the special characters
        female_name = re.sub(r'\W', ' ', str(name[sen]))
            
        # Dòng này dùng để loại bỏ Họ, Tên ra khỏi document (chỉ lấy middlename)
        female_name = female_name.split(' ',1)[1]
        female_name = female_name.rsplit(' ', 1)[0]
        female_name = female_name.strip()    
        # Converting to Lowercase
        female_name = female_name.lower()
        
            
        female_names = female_names + ' ' + female_name

    if (label[sen] == 0) and (len(str(name[sen]).split()) > 2):
    
        # Remove all the special characters
        male_name = re.sub(r'\W', ' ', str(name[sen]))
            
        # Dòng này dùng để loại bỏ Họ, Tên ra khỏi document
        male_name = male_name.split(' ',1)[1]
        male_name = male_name.rsplit(' ', 1)[0]
        male_name = male_name.strip()    
        # Converting to Lowercase
        male_name = male_name.lower()
        
            
        male_names = male_names + ' ' + male_name
        
        

In [14]:
male_names = male_names.split()
female_names = female_names.split()

#print(male_names)
#print(female_names)

['hiếu', 'việt', 'hùng', 'minh', 'đăng', 'an', 'phi', 'vũ', 'bảo', 'long', 'văn', 'nguyễn', 'anh', 'minh', 'long', 'hiếu', 'anh', 'khánh', 'tuấn', 'phú', 'viết', 'minh', 'khúc', 'tinh', 'xuân', 'quang', 'thanh', 'tuấn', 'bảo', 'huy', 'gia', 'minh', 'gia', 'minh', 'hoàng', 'hoàng', 'bảo', 'nhật', 'phụ', 'hoàng', 'nguyễn', 'quang', 'huy', 'hải', 'anh', 'khắc', 'đức', 'phùng', 'bảo', 'minh', 'đình', 'minh', 'trường', 'nguyễn', 'minh', 'sỹ', 'đình', 'vũ', 'bảo', 'việt', 'khôi', 'bình', 'hoàng', 'minh', 'minh', 'đức', 'nguyễn', 'minh', 'đắc']
['hoàng', 'quỳnh', 'hồng', 'thị', 'phương', 'hoàng', 'quỳnh', 'yến', 'tú', 'diệu', 'thị', 'vân', 'thanh', 'ngọc', 'hoàng', 'an', 'hà', 'bảo', 'hải', 'bảo', 'minh', 'ngọc', 'minh', 'phương', 'minh', 'bảo', 'nguyễn', 'ngân', 'vũ', 'an', 'gia', 'ngọc', 'phương', 'bảo', 'khánh', 'hoàng', 'mai', 'ngọc', 'anh', 'hoàng', 'thảo', 'trần', 'mỹ', 'minh', 'ngọc', 'châu', 'bảo', 'minh', 'hà', 'nguyễn', 'thanh', 'mỹ', 'minh', 'thảo', 'linh', 'nguyễn', 'tuệ', 'doãn',

In [15]:
#Tạo featureset là tất cả các từ trong middlename của male và female
featuresets = []

for i in range(0,len(female_names)):
    a = (female_names[i],'female')
    featuresets.append(a)
    
for i in range(0,len(male_names)):
    a = (male_names[i],'male')
    featuresets.append(a)
    
#print(featuresets)

In [16]:
import random 
random.shuffle(featuresets) 

In [17]:
#print(featuresets)

In [18]:
X = []
Y = []
for i in range(0,len(featuresets)):
    X.append(featuresets[i][0])
    Y.append(featuresets[i][1])
    
#print(X)
#print(Y)

In [19]:
len(Y)

133

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=False, max_features=1500, min_df=5, max_df=0.7)
X = vectorizer.fit_transform(X).toarray()

X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size = 0.4, random_state = 0)

In [21]:
# Xây dựng mô hình huấn luyện bằng Logistic Regression

from sklearn.linear_model import LogisticRegression
start = time.time()

classifier_LR = LogisticRegression(fit_intercept=True, n_jobs=4)
classifier_LR.fit(X_Train, Y_Train)

end = time.time()

print("Thời gian huấn Logistic Regression: ", str(end-start), " giây")

Thời gian huấn Logistic Regression:  2.3223659992218018  giây


In [22]:
# Xây dựng mô hình huấn luyện bằng NaiveBayes
from sklearn.naive_bayes import BernoulliNB

start = time.time()

classifier_NB = BernoulliNB(binarize=None)
classifier_NB.fit(X_Train, Y_Train)

end = time.time()

print("Thời gian huấn luyện Naive Bayes: ", str(end-start), " giây")

Thời gian huấn luyện Naive Bayes:  0.0031609535217285156  giây


In [23]:
from sklearn.ensemble import RandomForestClassifier

# Thêm biến này vào để kiểm tra thời gian huấn luyện
start = time.time()

classifier_RF = RandomForestClassifier(n_estimators=100, random_state=0)
classifier_RF.fit(X_Train, Y_Train) 

end = time.time()


print("Thời gian huấn luyện Random Forest: ", str(end-start), " giây")

Thời gian huấn luyện Random Forest:  0.28758835792541504  giây


In [24]:
# Xây dựng mô hình huấn luyện bằng KNN

from sklearn.neighbors import KNeighborsClassifier

start = time.time()

classifier_KNN = KNeighborsClassifier(n_neighbors=5)
classifier_KNN.fit(X_Train, Y_Train)

end = time.time()

print("Thời gian huấn luyện KNN: ", str(end-start), " giây")

Thời gian huấn luyện KNN:  0.005145549774169922  giây


In [25]:
# Predicting the test set results with LogisticRegresion
Y_Pred_LR = classifier_LR.predict(X_Test)

# Predicting the test set results with Naive Bayes
Y_Pred_NB = classifier_NB.predict(X_Test)

# Predicting the test set results with RF 
Y_Pred_RF = classifier_RF.predict(X_Test)

Y_Pred_KNN = classifier_KNN.predict(X_Test)

In [26]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print('Accuracy LogisticRegression: ', accuracy_score(Y_Test, Y_Pred_LR))
print('Accuracy Naive Bayes: ', accuracy_score(Y_Test,Y_Pred_NB))
print('Accuracy RF: ', accuracy_score(Y_Test, Y_Pred_RF))
print('Accuracy KNN: ', accuracy_score(Y_Test, Y_Pred_KNN))

Accuracy LogisticRegression:  0.5185185185185185
Accuracy Naive Bayes:  0.5185185185185185
Accuracy RF:  0.5
Accuracy KNN:  0.5185185185185185
